In [6]:
## Companion code for the article, "Kazhdan-Lusztig polynomials for \tilde{B}_2" by Karina Batistelli, Aram Bingham, and David Plaza

import time

# First we devine the polynomial ring where KL polynomials live, \Z[v,v^{-1}]
R.<v> = LaurentPolynomialRing(ZZ)
# The IwahoriHeckeAlgebra function will provide us the generators and relations for affine type B2
H = IwahoriHeckeAlgebra(['B',2,1], -v,v^(-1))  # the 1 after 'B', 2 indicates affine type
# The -v, v^{-1} specifies the quadratic relation our realization obeys.
# For T_s in the standard basis, T_s^2=(v^{-1}-v)T_s+1, as per "Intro to Soergel bimodules" (Elias et. al) (3.1)
# We rename the basis elements for convenience. 
# T: standard basis, C: canonical basis, Cp: the "other" self dual basis from [KL79] which we will not use
# C(x) corresponds to the Kazhdan-Lusztig basis as presented in Elias et. al
T=H.T(); C=H.C(); Cp=H.Cp()
# Affine Weyl group, the prefix means the default generators will be called s_0, s_1, s_2
W = WeylGroup(['B', 2, 1],prefix="s") 
[s0,s1,s2]= W.simple_reflections() # Let's shorten that to not have to write underscores

# Now we define the elements N_x from the article
def N(x):
    return sum([v**(x.length()-y.length())*T[y] for y in W.bruhat_interval(1,x)])

### now make the rewrite function
# We will use the articles notation where H_x is the standard basis element that Sage denotes as something like T[x]
def rwNH(h):    #rewrite the Hecke alg. elt. h in terms of N's and H's
    start=time.time()  # This is a clumsy way to keep track of time, but it works.
    hi=T(h)   #hi is for "h in" -- first convert h to standard basis, and we'll act on it peeling off N's
    step1=time.time()
    print('KL->Standard completed in '+str(step1-start)+' seconds.')
    expansion=[]  # where we will collect terms of expansion  
    while hi!=0: 
        terms=[str(w) for w in hi.terms()] # terms in sage's default order which is sort of by length but with some irregularities
        sup=[]   #support, will be list of elements of the form T[1,2,0], e.g in decreasing length order
        for x in terms:
            init="T"    
            sup.append(x[x.find(init):])  # This is a goofy way to strip the coefficients: 
            # We find where T appears in the stringified term and take everything including and after that
            # 
        sup.sort(key=len, reverse=True) # arranges T basis elts in decreasing length order (string length order and group length are compatible)
        #print(terms) 
        #print(sup) # for debugging
        
        
        index = [idx for idx, s in enumerate(terms) if sup[0] in s][0] # finds the list index of the longest elt in sages ordering
        ltb= sage_eval(sup[0], locals={'T':T}) # converts string back to algebra element, the longest in sup (lt is for "leading term")
        ltc=sage_eval(terms[index], locals={'T':T, 'v':v}).coefficients()[0] # now find coefficient of leading term
        #print(ltb)
        #print(hi.terms())
        #print(ltc)
        
        rwtime0=time.time()
        ltbrw = ltb.support()[0].reduced_word() # need lt underlying group elt as seq of coxeter gens)
        ltw=W.from_reduced_word(ltbrw) # this is the actual elt in the gp, not the expression
        rwtime1=time.time()
        #print('Reduced word extraction took '+str(rwtime1-rwtime0)+' seconds.')
        #ltc = lt.coefficients()[0]   #c is for 'coeff' 
        #print(ltc)
        
        ntime0=time.time()
        Nlt=N(ltw)  # take the N for the leading term
        ntime1=time.time()
        #print('N construction took '+str(ntime1-ntime0)+' seconds.') # in practice this took very little time, just wanted to see what were the costly steps
        #print(Nlt)
        
        stime0=time.time()
        if "-" not in str(hi-ltc*Nlt): #as long as we produce no negative symbols....
            hi -= ltc*Nlt   # subtract the N term (with coefficient) corresponding to leading term elt from the input
            expansion.append(str(ltc)+'*N'+str(ltbrw)) # keep track of this expansion
            #print(hi,"Subtracted N")
        else:
            hi -= ltc*ltb  # else we just subtract an H(x) with its coefficient and move on
            expansion.append(str(ltc)+'*T'+str(ltbrw))
            #print(hi,"Subtracted T")
        stime1=time.time()
       # print('Subtraction took '+str(stime1-stime0)+' seconds.')
        
    if hi==0:
        end=time.time()
        print('Success! Rewrite took '+ str(end-step1)+' seconds.' )
    else:
        print('Algo anda mal :(')
    print(*expansion, sep=' + ')
    
rwNH(N(s1*s2*s0*s2*s1*s2*s1)*C[0])



KL->Standard completed in 7.152557373046875e-07 seconds.
Success! Rewrite took 0.615220308303833 seconds.
1*N[1, 2, 0, 2, 1, 2, 1, 0] + v^2*N[1, 2, 1, 0]


In [7]:
rwNH(N(s1*s2*s0*s2*s1*s2*s1*s0*s2*s0)*C[1])


KL->Standard completed in 2.1457672119140625e-06 seconds.
Success! Rewrite took 3.8850955963134766 seconds.
1*N[1, 2, 0, 2, 1, 2, 1, 0, 2, 1, 0] + v*N[1, 2, 0, 2, 1, 2, 1, 0]


In [3]:
# If one wants to rewrite Hecke alg. elements only in terms of N's, the code below will work
# Note! These expansions are not unique! The algorithm given is greedy, so the output will depend on the order in which 
# the terms of standard basis expansion is fed to it.

import time


R.<v> = LaurentPolynomialRing(ZZ)
H = IwahoriHeckeAlgebra(['B',2,1], -v,v^(-1))
T=H.T(); Cp=H.Cp(); C=H.C()
W = WeylGroup(['B', 2, 1],prefix="s")
[s0,s1,s2]= W.simple_reflections()

# Same starting setup
def N(x):
    return sum([v**(x.length()-y.length())*T[y] for y in W.bruhat_interval(1,x)])

def rwN(h):    #rewrite in terms of N's
    start=time.time()
    hi=T(h)   #hi is for "h in" -- we'll act on it peeling off N's, but first we convert to standard basis
    step1=time.time()
    print('KL->Standard completed in '+str(step1-start)+' seconds.')
    expansion=[]  # where we will collect terms of expansion  
    while hi!=0:
        terms=[str(w) for w in hi.terms()] # terms in sage's default order
        sup=[]   # "support," will be list of elements of the form T[1,2,0], e.g in increasing length order
        for x in terms:
            start="T"
            sup.append(x[x.find(start):])
        sup.sort(key=len, reverse=True) # arranges T basis elts in increasing length order
        #print(terms)
        #print(sup)
        #print(sup[0])
        
        index = [idx for idx, s in enumerate(terms) if sup[0] in s][0] # finds the index of the longest elt in sages ordering
        ltb= sage_eval(sup[0], locals={'T':T}) # converts string back to algebra element, s[0] is the longest in sup
        ltc=sage_eval(terms[index], locals={'T':T, 'v':v}).coefficients()[0] # now find coefficient of leading term
        #print(ltb)
        #print(hi.terms())
        #print(ltc)
        
        rwtime0=time.time()
        ltbrw = ltb.support()[0].reduced_word() # need lt underlying group elt as seq of coxeter gens)
        ltw=W.from_reduced_word(ltbrw) # this is the actual elt in the gp, not the expression
        rwtime1=time.time()
        #print('Reduced word extraction took '+str(rwtime1-rwtime0)+' seconds.')
        #ltc = lt.coefficients()[0]   #c is for 'coeff' (lists are decreasing in length/bruhat it seems)
        #print(ltc)
        
        ntime0=time.time()
        Nlt=N(ltw)  # take the N for the leading term
        ntime1=time.time()
        #print('N construction took '+str(ntime1-ntime0)+' seconds.')
        #print(Nlt)
        
        stime0=time.time()
        hi -= ltc*Nlt   # and subtract it from the input
        expansion.append(str(ltc)+'*N'+str(ltbrw))
        stime1=time.time()
        #print('Subtraction took '+str(stime1-stime0)+' seconds.')
        
    end=time.time()
    print('Success! Rewrite took '+ str(end-step1)+' seconds.' )
    print(*expansion, sep=' + ')    

# We also define the elements D_x^y as defined in Section 6 of the article
def D(x,y):
    return sum([v**(x.length()-z.length())*T[z] for z in W.bruhat_interval(1,x) if not z.bruhat_le(y)]) 

# This function defines the "prime" involution that swaps s1 and s0
def fi(x):
    s=list(str(x)) # takes an element of the form e.g. s2*s1*s0*s2*s1*s2*s1*s0*s1
    t=['0' if y=='1' else '1' if y=='0' else y for y in s] # once symbols are in a list, we swap 1's and 0's
    t=sage_eval(''.join(t), locals={'s1':s1, 's2':s2, 's0':s0, 'C':C, 'T':T}) #recover the associated element
    return t 

def Dtil(x): # \tilde{D}(x) is D_x^{x'}, which is used in the definition of U_x
    return D(x,fi(x))

# U_x from Section 6
def U(x):
    return N(x)+D(fi(x),x)

# For x of length one, the associated canonical basis element has this form; see Elias et al, ch. 3
def Hb(x):
    return T[x]+v 

# Elements x_n of the thick region as defined in the paper
def x(n):
    frag=[s1,s2,s1,s0,s2,s0] #repeating subsequence
    reps=ceil(n/6)
    enough=frag*reps #extend the repeating segment far enough
    return prod(i for i in enough[:n]) # then take the first n symbols and multiply

# x'_n, could have been defined using fi(x) but instead we define analagous to x(n)
def xp(n):
    frag=[s0,s2,s0,s1,s2,s1]
    reps=ceil(n/6)
    enough=frag*reps
    return prod(i for i in enough[:n])

# Elements e_n and e'_n of the other branches of the thick region
def e(n):
    return s1*xp(n)

def ep(n):
    return s0*x(n)

# Elements d_n of the thin region as defined in the paper
def d(n):
    frag=[s2,s1,s2,s0]
    reps=ceil(n/4)
    enough=frag*reps
    return prod(i for i in enough[:n])

# d'_n
def dp(n):     
    frag=[s2,s0,s2,s1]
    reps=ceil(n/4)
    enough=frag*reps
    return prod(i for i in enough[:n])

# theta's of the big region as defined in the paper
def theta(m,n):
    if m%2==0:
        return x(3*(m+1))*dp(4*n+1)
    else:
        return x(3*(m+1))*d(4*n+1)

# \theta'(m,n) (apply the prime involution)
def thetap(m,n):
    if m%2==0:
        return xp(3*(m+1))*d(4*n+1)
    else:
        return xp(3*(m+1))*dp(4*n+1)    

# According to results of the paper, we can define the canconical basis elements associated to theta's and "friends"
# directly using the N elements. Hb is for "H bar", the notation used in the paper.
def HbTheta(m): # This is for \theta(m,0)
    return sum([v^(2*i)*N(theta(m-2*i,0)) for i in range(floor(m/2)+1)])

# H bar for theta prime
def HbThetap(m): #\theta'(m,0)
    return sum([v^(2*i)*N(thetap(m-2*i,0)) for i in range(floor(m/2)+1)])

# According to Theorem 1.1, Hb_{s_2*s_1*\theta'(m,0)} = Hb_{s_2}Hb_{s_1}Hb_{\theta'(m,0)} -Hb_{\theta'(m,0)} 
# We would sometimes call s_1=s, s_2=t, s_0=u during the course of our work
def Hbtsthetap(m):
    return Hb(2)*Hb(1)*HbThetap(m)-HbThetap(m)

# premultiply \theta(m,0) by s_1s_2s_0
def HbstuTheta(m):
    return((Hb(1)*Hb(2)*Hb(0)-Hb(0)-v^(-1)-v)*Hbtheta(m))

# now with t_m multiplied on the right (the element that makes \theta(m,0) grow)
def HbstuThetat_m(m):
    if m%2==0:
        return(HbstuTheta(m)*Hb(0))
    else:
        return(HbstuTheta(m)*Hb(1))

#similar ideas  

def HbsThetat_m(m): # for \theta(m,0)t_m
    return(Hb(1)*HbThetap(m)*Hb((1+m)%2))

def HbThetat_ms2(m): # for \theta(m,0)t_m s_2
    if m%2==0:
        return(HbTheta(m)*Hb(0)*Hb(2)-HbTheta(m))
    else:
        return(HbTheta(m)*Hb(1)*Hb(2)-HbTheta(m))

def Hbs0theta(m): # for s_0\theta(m,0)
    return(Hb(0)*HbTheta(m))

#Supp(m,n) as defined in the introduction of the paper
def S(m,n):
    if m%2!=0:
        return {(m-2*i,n-j) for i in range(floor(m/2)+1) for j in range(n+1)}
    else:
        return {(m-2*i,n-j) for i in range(floor(m/2)) for j in range(n+1)}.union({(0,n-2*i) for i in range(floor(n/2)+1)})

#Direct description of canonical basis elts for thetas as per Theorem 1.1
def Hbtheta(m,n):
    return sum([v^((m-a)+2*(n-b))*N(theta(a,b)) for (a,b) in S(m,n)])

# Now we go to the rest of Theorem 1.1. l and r will represent the x and y terms in that theorem
def Hbamigo(m,n,l,r): # 'amigos de theta' are the elements corresponding to alcoves that are either neighbors in the fundamental chamber, 
# or the corresponding alcoves in the other chambers of the big region of these
    if l==1: #multiply on left by identity
        if r==1: # on right by identity
            return Hbtheta(m,n) # no change
        elif r==s0 or r==s1: # right multiply by t_m, etc.
            return Hbtheta(m,n)*C[r]
        elif r==s0*s2 or r==s1*s2:
            return Hbtheta(m,n)*C[r]-Hbtheta(m,n)
        elif r==s0*s2*s1:
            return Hbamigo(m,n,1,s0*s2)*C[1]-Hbtheta(m,n)*C[0]
        elif r==s1*s2*s0:
            return Hbamigo(m,n,1,s1*s2)*C[0]-Hbtheta(m,n)*C[1]
        else:
            print('Bad input, try again.')
    elif l==s0:
        return Hb(0)*Hbamigo(m,n,1,r)
    elif l==s2*s0:
        return Hb(2)*Hbamigo(m,n,s0,r)-Hbamigo(m,n,1,r)
    elif l==s1*s2*s0:
        return Hb(1)*Hbamigo(m,n,s2*s0,r)-Hbamigo(m,n,s0,r)
    else:
        print('Bad input, try again.')

# This tool can help us see the explicit form of the friends of thetas
# For example:
for m in range(6):
    rwN(Hbamigo(m,0,s0,1))

KL->Standard completed in 4.76837158203125e-07 seconds.
Success! Rewrite took 0.11400485038757324 seconds.
1*N[0, 2, 1, 2, 1]
KL->Standard completed in 1.1920928955078125e-06 seconds.
Success! Rewrite took 0.9381084442138672 seconds.
1*N[0, 2, 1, 2, 1, 0, 2, 0] + v*N[1, 2, 0, 2, 0]
KL->Standard completed in 1.6689300537109375e-06 seconds.
Success! Rewrite took 1.6780059337615967 seconds.
1*N[0, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1] + v*N[1, 2, 0, 2, 1, 0, 2, 1] + v^2*N[0, 2, 1, 2, 1]
KL->Standard completed in 1.430511474609375e-06 seconds.
Success! Rewrite took 3.894035577774048 seconds.
1*N[0, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 0] + v*N[1, 2, 0, 2, 1, 0, 2, 1, 0, 2, 0] + v^2*N[0, 2, 1, 2, 1, 0, 2, 0] + v^3*N[1, 2, 0, 2, 0]
KL->Standard completed in 2.86102294921875e-06 seconds.
Success! Rewrite took 8.48439908027649 seconds.
1*N[0, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1] + v*N[1, 2, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1] + v^2*N[0, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1] + v^3*N[1, 2, 0, 2, 1, 0,

In [4]:
# Careful! Already at length 16 this may take quite a long time.
T(C[2,1,2,0,2,1,2,0,2,1,2,0,2,1,2,0])

In [ ]:
# The costly step is not the conversion of the canonical basis element, but its computation
rwN(T(C[2,1,0,2]))

KL->Standard completed in 1.1920928955078125e-06 seconds.
Success! Rewrite took 0.20458483695983887 seconds.
1*N[2, 1, 0, 2] + v*N[2]
